In [ ]:
%store -r
%store

## Create solutions

In [14]:
import time
from time import sleep
import json
import pandas as pd
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
pd.set_option('display.max_colwidth', 200)  
import boto3

In [15]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [16]:
recipes = personalize.list_recipes()['recipes']

In [17]:
recipes_df = pd.DataFrame(recipes)
recipes_df[['name','recipeArn']]

,name,recipeArn
0,aws-hrnn,arn:aws:personalize:::recipe/aws-hrnn
1,aws-hrnn-coldstart,arn:aws:personalize:::recipe/aws-hrnn-coldstart
2,aws-hrnn-metadata,arn:aws:personalize:::recipe/aws-hrnn-metadata
3,aws-personalized-ranking,arn:aws:personalize:::recipe/aws-personalized-ranking
4,aws-popularity-count,arn:aws:personalize:::recipe/aws-popularity-count
5,aws-sims,arn:aws:personalize:::recipe/aws-sims
6,aws-user-personalization,arn:aws:personalize:::recipe/aws-user-personalization


### User Personalization
La receta de personalización de usuario está optimizada para todos los escenarios de recomendación de USER_PERSONALIZATION. Al recomendar elementos, utiliza la exploración automática de elementos.

Con la exploración automática, Amazon Personalize prueba automáticamente diferentes recomendaciones de artículos, aprende de cómo los usuarios interactúan con estos artículos recomendados y aumenta las recomendaciones de artículos que generan una mejor participación y conversión. Esto mejora el descubrimiento y la participación de los artículos cuando tiene un catálogo que cambia rápidamente o cuando los artículos nuevos, como artículos de noticias o promociones, son más relevantes para los usuarios cuando están frescos.

Puede equilibrar cuánto explorar (donde los elementos con menos interacciones, datos o relevancia se recomiendan con más frecuencia) con cuánto explotar (donde las recomendaciones se basan en lo que sabemos o en la relevancia). Amazon Personalize ajusta automáticamente las recomendaciones futuras en función de los comentarios implícitos de los usuarios.

Primero, seleccione la receta buscando el ARN en la lista de recetas anterior.

In [18]:
user_personalization_recipe_arn = 'arn:aws:personalize:::recipe/aws-user-personalization'

#### Crear la solución

La solución hace referencia a la combinación de una receta de Amazon Personalize (https://aws.amazon.com/es/personalize/), parámetros personalizados y una o más versiones de la solución.

Una vez creada la Version Solution, la solución con mejor desempeño, creas la campaña y obtienes las recomendaciones.

In [ ]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "personalize-anime-userpersonalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_solution_arn = user_personalization_create_solution_response['solutionArn']
print(json.dumps(user_personalization_solution_arn, indent=2))

#### Create the solution version

Una vez que tenga una solución, debe crear una versión para completar el entrenamiento del modelo. La capacitación puede tardar un tiempo en completarse, más de 25 minutos y un promedio de 90 minutos para esta receta con nuestro conjunto de datos. Normalmente, usaríamos un ciclo while para sondear hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas, y el objetivo aquí es crear muchos modelos e implementarlos rápidamente. Así que configuraremos el ciclo while para todas las soluciones más abajo en el cuaderno. Allí también encontrará instrucciones para ver el progreso en la consola de AWS.

In [ ]:
userpersonalization_create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)
userpersonalization_solution_version_arn = userpersonalization_create_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_create_solution_response, indent=2))

### SIMS


SIMS es uno de los algoritmos más antiguos utilizados en Amazon para los sistemas de recomendación. Un caso de uso central es cuando tiene un elemento y desea recomendar elementos con los que se ha interactuado de manera similar en toda su base de usuarios. Esto significa que el resultado no es personalizado por usuario. A veces, esto lleva a recomendar artículos en su mayoría populares, por lo que hay un hiperparámetro que se puede ajustar y reducirá los artículos populares en los resultados.

Para nuestro caso de uso, utilizando los datos de Animes, supongamos que elegimos un Anime en particular. Luego podemos usar SIMS para recomendar otros anime en función del comportamiento de interacción de toda la base de usuarios. Los resultados no son personalizados por usuario, sino que difieren según el anime que elegimos como entrada.

Al igual que la última vez, comenzamos seleccionando la receta.

#### Crear la solución

Al igual que con HRNN, comience creando primero la solución. Aunque proporcione el ARN del conjunto de datos en este paso, el modelo aún no está entrenado. Vea esto como un identificador en lugar de un modelo entrenado.

In [21]:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

In [ ]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-anime-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

#### Crear la version de la solución

In [ ]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

### Personalized Ranking

En lugar de simplemente recomendar lo que es más probable para el usuario en cuestión, este algoritmo también incluye un usuario y una lista de elementos que se representan en el orden de relevancia más probable para el usuario. El caso de uso aquí es para filtrar en categorías únicas en las que no tiene metadatos de elementos para crear un filtro, o cuando tiene una colección amplia que le gustaría ordenar mejor para un usuario en particular.


In [24]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

In [ ]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-anime-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

In [ ]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

### Ver el estado de la solucion en la consola: 

* Busca en la consola "Amazon Personalize"
* Haga clic en `Dataset gorups`.
* Haga clic en el nombre de su grupo de conjuntos de datos, probablemente algo con POC en el nombre.
* Haga clic en `Soluciones y recetas`.
* Ahora verá una lista de todas las soluciones que creó anteriormente, incluida una columna con el estado de las versiones de la solución. Una vez que está `Activo`, su solución está lista para ser revisada. También es capaz de ser desplegado.

O simplemente ejecuta la celda a continuación para realizar un seguimiento del estado de creación de la versión de la solución.

In [ ]:
in_progress_solution_versions = [
    userpersonalization_solution_version_arn,
    sims_solution_version_arn,
    rerank_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

## Evaluate solution versions <a class="anchor" id="eval"></a>
[Back to top](#top)

No debería tomar más de una hora entrenar todas las soluciones de este cuaderno. Mientras el entrenamiento está en progreso, recomendamos tomarse el tiempo para leer en detalle los diversos algoritmos (recetas) y su comportamiento. Este también es un buen momento para considerar alternativas sobre cómo se ingresaron los datos en el sistema y qué tipo de resultados espera ver.


Recomendamos leer la siguiene documentación [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) para entender las metricas. 

Aca un breve resumen: 

* *Relevant recommendation* se refiere a una recomendación que coincide con un valor en los datos de prueba para el usuario en particular.
* *Rank* se refiere a la posición de un elemento recomendado en la lista de recomendaciones. Se supone que la posición 1 (la parte superior de la lista) es la más relevante para el usuario.
* *Query* hace referencia al equivalente interno de una llamada GetRecommendations.

Las metricas entregadas por Personalize son: 

* **coverage**: Es el porcentaje de los items que el modelo está recomendando en general (incluye ambos datasets Items y interacciones). Es decir por ejemplo en SIMS, tiene un coverage de 45% es decir hay un 55% de items que no están siendo recomendados.
* **mean_reciprocal_rank_at_25**: El [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)es la primera recomendación relevante de las 25 recomendaciones principales sobre todas las consultas. Esta métrica es apropiada si está interesado en la recomendación individual mejor calificada.
* **normalized_discounted_cumulative_gain_at_K**: La ganancia descontada asume que las recomendaciones más bajas en una lista de recomendaciones son menos relevantes que las recomendaciones más altas. Por lo tanto, cada recomendación se descuenta (se le da un peso menor) por un factor que depende de su posición. Para generar la [ganancia acumulada descontada](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) en K, se suma cada recomendación descontada relevante en las recomendaciones K principales. La ganancia acumulada descontada normalizada (NDCG, por sus siglas en inglés) es el DCG dividido por el DCG ideal, de modo que el NDCG está entre 0 y 1. (El DCG ideal es donde las K recomendaciones principales se ordenan por relevancia). Amazon Personalize utiliza un factor de ponderación de 1/. log(1 + posición), donde la parte superior de la lista es la posición 1. Esta métrica recompensa los elementos relevantes que aparecen cerca de la parte superior de la lista, porque la parte superior de una lista suele llamar más la atención.
* **precision_at_K**: El número de recomendaciones relevantes de las K recomendaciones principales divididas por K. Esta métrica recompensa la recomendación precisa de los elementos relevantes.


Veamos el resultado de la evaluación de nuestras versiones. 

In [ ]:
user_personalization_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = userpersonalization_solution_version_arn
)['metrics']
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)['metrics']

rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)['metrics']

user_personalization_solution_metrics_response['solution'] = 'user personalization'
sims_solution_metrics_response['solution'] = 'similar items'
rerank_solution_metrics_response['solution'] = 're rank'


In [55]:
import pandas as pd
metrics_df = pd.DataFrame([user_personalization_solution_metrics_response, sims_solution_metrics_response,
                           rerank_solution_metrics_response]).set_index('solution')

metrics_df.rename(columns = {'mean_reciprocal_rank_at_25':'MRR_25', 'normalized_discounted_cumulative_gain_at_10':'NDCG_10', 
                             'normalized_discounted_cumulative_gain_at_25':'NDCG_15',
                              'normalized_discounted_cumulative_gain_at_5':'NDCG_5', 'precision_at_10': 'P_10' , 'precision_at_25': 'P_25' ,
                             'precision_at_5': 'P_5' }, inplace = True) 
metrics_df

,coverage,MRR_25,NDCG_10,NDCG_15,NDCG_5,P_10,P_25,P_5
solution,,,,,,,,
user personalization,0.1782,0.5750,0.4652,0.5642,0.4054,0.2049,0.1308,0.2770
similar items,0.4498,0.1822,0.1900,0.2580,0.1494,0.0535,0.0408,0.0644
re rank,0.1899,0.5837,0.4717,0.5708,0.4129,0.2079,0.1321,0.2820


In [28]:
%store userpersonalization_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn
%store user_personalization_solution_arn
%store sims_solution_arn
%store rerank_solution_arn

Stored 'userpersonalization_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)
Stored 'user_personalization_solution_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'rerank_solution_arn' (str)


In [ ]:
%store